# **DS501 Class Project**
**Authors:** Daniel Alvarado Segura and Jun Lee

**Description**: In this project we plan to merge two data sets which include housing selling prices for California. With this new merged data set we will be able to find a much more refined image of the house market in the area of California. 

**Data Sets**: We will use two open source data sets which inlcude information about the housing market of the area of California. The first set contains information of the houses per quadrant based on longitude and latitude. The second set has information of specific houses and their specific address. 

# **Data Loading and Cleaning**
**Process:** To be able to merge this two data bases we msut first clean the data by dropping any rows with key information such as its address/location and house features such as number of rooms and proximity to the coast. Additionally, the second date set is split between a training set and a testing set. We will concatenate the contents of these two, to then be able to merge it with the first data set

**Data Loading of First Data Set**

In [ ]:
#Data Loading and Cleaning of the first set
import pandas as pd

#Retrieve data from csv
coordinate_based_data = pd.read_csv("file_address") #Use local path data to file

#Clean rows with empty data for Address and Bedrooms
cleaned_coordinate_based_data = coordinate_based_data.dropna(subset=['longitude', 'latitude'])

#Check data
print(f"Full data:\n{cleaned_coordinate_based_data}")

**Data Loading of Second Data Set**

In [ ]:
#Data Loading and Cleaning of the second set
import pandas as pd

#Retrieve both parts of data set
train_data = pd.read_csv("data addres")
test_data = pd.read_csv("data addres")

#Join both sets to have complete set
full_data = pd.concat([train_data, test_data], axis = 0)

#Clean rows with empty data for Address and Bedrooms
full_data = full_data.dropna(subset=['Address', 'Bedrooms'])

#Check final data
print(f"Full data:\n{full_data}")